In [ ]:
# import important libraries

import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras


### **Data Generation**

In [ ]:
### projectile motion equations

### Horizontal motion equations for the projectile
### x_f = x_i + v_x * delta_t
### v_fx = v_ix
### v_fy = v_iy + a_y * delta_t
### y_f = y_i + v_iy * t - 1/2 * g * delta_t**2
### v_fy**2 = v_iy**2 - 2*g*(y_f - y_i)

# Generating Mock Data for Training Data

# number of mock points
num_points = 10**6;

g = 9.81; # gravity

x_i = 0; # by convention of coordinate system
y_i = 0; # by convention of coordinate system

def generate_data(num_samples):
    theta = np.random.uniform(0, np.pi/2, num_samples)  # Only 0-90° makes physical sense
    v_i = np.random.uniform(0, 80, num_samples)  # 0-80 mph
    R = (v_i**2 * np.sin(2*theta)) / g  # Range calculation
    return np.column_stack([v_i, theta]), R  # Features and labels


In [ ]:
x_train, y_train = generate_data(10**6)
x_test, y_test = generate_data(10**5)

In [ ]:
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)

### **Neural Network**

In [ ]:
x_train = np.load('x_train.npy', allow_pickle = True)
y_train = np.load('y_train.npy', allow_pickle = True)
x_test = np.load('x_test.npy', allow_pickle = True)
y_test = np.load('y_test.npy', allow_pickle = True)

In [ ]:
print(x_train, y_train)
print(x_test,y_test)

[[69.55524748  1.05929261]
 [14.06780892  1.16464443]
 [77.92618098  0.99104145]
 ...
 [30.76444721  1.11298803]
 [41.44462789  0.14664681]
 [53.9774375   0.10101746]] [421.00277972  14.64350322 567.38918373 ...  76.50127821  50.62041916
  59.59686605]
[[ 9.68869376  0.32456376]
 [20.17743202  1.56923767]
 [67.90804485  0.65110141]
 ...
 [19.36911282  0.98541847]
 [52.67585076  1.35764627]
 [63.75860152  1.06929619]] [5.78431220e+00 1.29372896e-01 4.53227073e+02 ... 3.52234087e+01
 1.16959298e+02 3.49366772e+02]


In [ ]:
# Normalize data
v_mean, v_std = x_train[:,0].mean(), x_train[:,0].std()
theta_mean, theta_std = x_train[:,1].mean(), x_train[:,1].std()

x_train[:,0] = (x_train[:,0] - v_mean) / v_std
x_train[:,1] = (x_train[:,1] - theta_mean) / theta_std
x_test[:,0] = (x_test[:,0] - v_mean) / v_std
x_test[:,1] = (x_test[:,1] - theta_mean) / theta_std


In [ ]:
# Model for regression
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    # Simple dense network works better for this
    x = keras.layers.Dense(64, activation='relu')(input_layer)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dense(64, activation='relu')(x)

    # Single output for regression (no activation)
    output_layer = keras.layers.Dense(1)(x)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Input shape is (2,) for [v_i, theta]
model = make_model(input_shape=(2,))

In [ ]:
model.compile(
    optimizer='adam',
    loss='mse',  # Mean squared error for regression
    metrics=['mae']  # Mean absolute error
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 85s 3ms/step - loss: 1338.1565 - mae: 9.3937 - val_loss: 0.1750 - val_mae: 0.3145
Epoch 2/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 90s 3ms/step - loss: 0.8844 - mae: 0.5749 - val_loss: 0.2345 - val_mae: 0.3499
Epoch 3/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 137s 3ms/step - loss: 0.7720 - mae: 0.5171 - val_loss: 0.5482 - val_mae: 0.5056
Epoch 4/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 86s 3ms/step - loss: 0.6769 - mae: 0.4972 - val_loss: 1.0339 - val_mae: 0.6628
Epoch 5/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 138s 3ms/step - loss: 0.6981 - mae: 0.4960 - val_loss: 0.4544 - val_mae: 0.4571
Epoch 6/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 142s 3ms/step - loss: 0.6367 - mae: 0.4713 - val_loss: 0.0942 - val_mae: 0.2403
Epoch 7/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 89s 3ms/step - loss: 0.6454 - mae: 0.4694 - val_loss: 1.2396 - val_mae: 0.7676
Epoch 8/10
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 0.6312 - mae: 0.4652 - val_loss: 0.7476 - val_mae: 0.6294
Epoch 9/1